# Walmart Sales Prediction (M5 Competition)

For large supermarkets like Walmart stores, forecasting future sales of products is crucial for keeping stock such that consumer demand can be met. This forecasting study focuses on the demand for a subcategory of hobby products in a Walmart store in California, USA. To be exact, we will try to forecast the need for 149 hobby products for 28 consecutive days. 

Along with previous demand for the products, we have access to data about the sales prices of the products and special events on the calendar for the sampled time series data. We hope to give insight into the accuracy of models in product sales forecasting by comparing the performance of traditional time series forecasting and machine learning methods.


In [62]:
# Import Relevent Packages

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from prophet import Prophet
import time
import warnings
from itertools import cycle
from sklearn.svm import SVR
import statsmodels.api as sm
from pmdarima import auto_arima
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing

In [20]:
%matplotlib inline
plt.style.use('bmh')
sns.set_style("darkgrid")
plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)
warnings.filterwarnings("ignore")
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [50]:
# Read in data
os.chdir("../data") 

merged = pd.read_csv('merged.csv')
merged.date = pd.to_datetime(merged.date)
merged.head()

,id,day,demand,date,wm_yr_wk_x,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,item_id,sell_price
0,HOBBIES_2_002_CA_3_validation,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,HOBBIES_2_002,1.97
1,HOBBIES_2_002_CA_3_validation,d_2,0,2011-01-30,11101,Sunday,2,1,2011,NaN,NaN,NaN,NaN,0,HOBBIES_2_002,1.97
2,HOBBIES_2_002_CA_3_validation,d_3,0,2011-01-31,11101,Monday,3,1,2011,NaN,NaN,NaN,NaN,0,HOBBIES_2_002,1.97
3,HOBBIES_2_002_CA_3_validation,d_4,1,2011-02-01,11101,Tuesday,4,2,2011,NaN,NaN,NaN,NaN,1,HOBBIES_2_002,1.97
4,HOBBIES_2_002_CA_3_validation,d_5,0,2011-02-02,11101,Wednesday,5,2,2011,NaN,NaN,NaN,NaN,1,HOBBIES_2_002,1.97


## 3.1 Forecas with Prophet

The Prophet is a forecasting package launched by Facebook to predict time series data based on an additive model. While it does not take attributes other than the prior sales data, it fits with non-linear trends and various seasonality, including holiday effects. It also demonstrates remarkable forecasting power even with missing values and outliers.

In [53]:
# Extract unique id names

id_list = list(merged.id.unique())
len(id_list)

149

In [59]:
# Create a for loop to run all the 149 items

result = pd.DataFrame()

for item in id_list:
    mini = merged[merged.id ==item][["date", "demand"]]
    mini.rename(columns={'date':'ds', 'demand':'y'}, inplace = True)

    m = Prophet(daily_seasonality=True, yearly_seasonality=True)
    m.fit(mini)
    future = m.make_future_dataframe(periods=28, include_history=False)
    forecast = m.predict(future)[['ds', 'yhat']]
    forecast["id"] = item
    result = pd.concat([result, forecast])
    
result.head()

,ds,yhat,id
0,2016-06-20,0.036227,HOBBIES_2_002_CA_3_validation
1,2016-06-21,0.042660,HOBBIES_2_002_CA_3_validation
2,2016-06-22,0.063581,HOBBIES_2_002_CA_3_validation
3,2016-06-23,0.070406,HOBBIES_2_002_CA_3_validation
4,2016-06-24,0.098600,HOBBIES_2_002_CA_3_validation


In [60]:
# Reframe it to the submission format

wide_format= result.pivot(index="id", columns="ds", values="yhat").reset_index()
wide_format = wide_format.rename_axis(None, axis=1)
wide_format.head()

#wide_format.to_csv("sub_prophet.csv")

,id,2016-06-20 00:00:00,2016-06-21 00:00:00,2016-06-22 00:00:00,2016-06-23 00:00:00,2016-06-24 00:00:00,2016-06-25 00:00:00,2016-06-26 00:00:00,2016-06-27 00:00:00,2016-06-28 00:00:00,2016-06-29 00:00:00,2016-06-30 00:00:00,2016-07-01 00:00:00,2016-07-02 00:00:00,2016-07-03 00:00:00,2016-07-04 00:00:00,2016-07-05 00:00:00,2016-07-06 00:00:00,2016-07-07 00:00:00,2016-07-08 00:00:00,2016-07-09 00:00:00,2016-07-10 00:00:00,2016-07-11 00:00:00,2016-07-12 00:00:00,2016-07-13 00:00:00,2016-07-14 00:00:00,2016-07-15 00:00:00,2016-07-16 00:00:00,2016-07-17 00:00:00
0,HOBBIES_2_001_CA_3_validation,0.159759,0.137535,0.129654,0.136289,0.143165,0.106754,0.154174,0.134505,0.115026,0.110943,0.122331,0.134789,0.104636,0.158816,0.146218,0.133921,0.136926,0.155104,0.173857,0.149320,0.208272,0.199459,0.189845,0.194345,0.212778,0.230529,0.203747,0.259261
1,HOBBIES_2_002_CA_3_validation,0.036227,0.042660,0.063581,0.070406,0.098600,0.078797,0.099433,0.080780,0.086297,0.105361,0.109370,0.133792,0.109293,0.124367,0.099369,0.097864,0.109359,0.105401,0.121627,0.088880,0.095841,0.063058,0.054284,0.059211,0.049556,0.061113,0.024859,0.029586
2,HOBBIES_2_003_CA_3_validation,0.606129,0.545477,0.705462,0.674986,0.595710,0.818531,0.910145,0.636310,0.564901,0.713713,0.671966,0.581647,0.793971,0.875938,0.593587,0.515048,0.658326,0.612813,0.520620,0.733029,0.817057,0.538705,0.466006,0.616821,0.580346,0.498454,0.722150,0.818145
3,HOBBIES_2_004_CA_3_validation,-0.047937,-0.084226,0.056323,0.040529,-0.013401,0.047323,0.053136,-0.075786,-0.103722,0.044781,0.036417,-0.010707,0.056115,0.067259,-0.057139,-0.081371,0.070020,0.063755,0.017988,0.085486,0.096701,-0.028141,-0.053237,0.096970,0.089307,0.042025,0.107990,0.117743
4,HOBBIES_2_005_CA_3_validation,0.102767,0.137736,0.141471,0.119157,0.133265,0.126196,0.129707,0.092165,0.126663,0.129731,0.106560,0.119638,0.111393,0.113613,0.074709,0.107822,0.109538,0.085107,0.097079,0.087943,0.089549,0.050364,0.083582,0.085833,0.062401,0.075862,0.068715,0.072811


## 3.2 Using Machine Learing methods with feature engineering

Introducing appropriate variables can increase models’ complexity and ameliorate underfitting. In that sense, feature engineering is the critical process of selecting relevant features and applying a transformation to these data to construct a robust predictive model \cite{Kuhn19}.

In the competition, two major features regarding time series are applied to enrich the dataset. First, based on the insight acquired from data visualization, we assume the demand for each item is autocorrelated to seven days ago. Therefore, a lag of seven days in demand is introduced. The second assumption is that a similar sales pattern could appear on both an annual and weekly basis. We consequently utilize a groupby method to add descriptive statistics for each month and day of the week. Several new features are generated as ['lag\_7', 'rmean\_7\_7', 
'demand\_month\_mean', 'demand\_month\_max', 'demand\_month\_max\_to\_min\_diff', 'demand\_dayofweek\_mean', 'demand\_dayofweek\_median', 'demand\_dayofweek\_max']. 
Figure 8 illustrates the above-mentioned feature transformation for a single item. 

Besides, as categorical features require additional encoding to fit into regression models, several approaches are also experimented with, including one-hot encoding, label encoding, mean encoding, and group-by encoding. In our final practice, the categorical factors, including event names and event types, are processed with label encoder as it outperforms others. Figure 9 demonstrates the categorical factors after label encoding.

In [58]:
def lags_windows(df):
    lags = [7]
    lag_cols = ["lag_{}".format(lag) for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        df[lag_col] = df[["id","demand"]].groupby("id")["demand"].shift(lag)
        
    wins = [7]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            df["rmean_{}_{}".format(lag,win)] = df[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())  
    return df

def per_timeframe_stats(df, col):
    months = df['month'].unique().tolist()
    for y in months:
        df.loc[df['month'] == y, col+'_month_mean'] = df.loc[df['month'] == y].groupby(['id'])[col].transform(lambda x: x.mean()).astype("float32")
        df.loc[df['month'] == y, col+'_month_max'] = df.loc[df['month'] == y].groupby(['id'])[col].transform(lambda x: x.max()).astype("float32")
        df.loc[df['month'] == y, col+'_month_min'] = df.loc[df['month'] == y].groupby(['id'])[col].transform(lambda x: x.min()).astype("float32")
        df[col + 'month_max_to_min_diff'] = (df[col + '_month_max'] - df[col + '_month_min']).astype("float32")

    dayofweek = df['wday'].unique().tolist()

    for y in dayofweek:
        df.loc[df['wday'] == y, col+'_dayofweek_mean'] = df.loc[df['wday'] == y].groupby(['id'])[col].transform(lambda x: x.mean()).astype("float32")
        df.loc[df['wday'] == y, col+'_dayofweek_median'] = df.loc[df['wday'] == y].groupby(['id'])[col].transform(lambda x: x.median()).astype("float32")
        df.loc[df['wday'] == y, col+'_dayofweek_max'] = df.loc[df['wday'] == y].groupby(['id'])[col].transform(lambda x: x.max()).astype("float32")
    
    return df

def feat_eng(df):
    df = lags_windows(df)
    df = per_timeframe_stats(df,'demand')
    
    return df

In [77]:
def preprocess_item(dataframe, item_id):

    this_item = dataframe[dataframe.id ==item_id]
    this_item['day'] = this_item['day'].apply(lambda x: x.split('_')[1]).astype(int)
    this_item = this_item.drop(['weekday', 'item_id'], axis = 1)
    this_item = this_item.fillna('No')

    for c in ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']:
        this_item[c] = LabelEncoder().fit_transform(this_item[c])

    this_train = this_item[this_item['date'] <= '2016-05-22']
    this_test = this_item[(this_item['date'] > '2016-05-6') & (this_item['date'] <= '2016-06-19')]
    this_train = feat_eng(this_train)
    this_train = this_train.dropna()
    
    return this_train, this_test

def predict_sales(test, model, train_cols, pred_name, tst, day):
        
    tst_X = tst.loc[tst.date == day , train_cols].copy()
    tst_X = tst_X.fillna(0) 
    test.loc[test.date == day, pred_name] = model.predict(tst_X)

    return test

In [ ]:
sub_final = pd.DataFrame()

for item_id in id_list:
    
    # Split the data
    this_train, this_test = preprocess_item(merged, item_id)
    
    predictions = pd.DataFrame()
    predictions['date'] = this_test['date']
    
    # Choose features to use
    useless_cols = ['id','item_id','demand','date','weekday','demand_month_min', 'day']
    linreg_train_cols = ['sell_price','year','month','wday','lag_7','rmean_7_7']
    
    train_cols = this_train.columns[~this_train.columns.isin(useless_cols)]
    X_train = this_train[train_cols].copy()
    y_train = this_train["demand"]

    # Fit in models
#    m_linreg = LinearRegression().fit(X_train[linreg_train_cols], y_train)
#    m_rf = RandomForestRegressor(n_estimators=50, max_depth=5, random_state=26, n_jobs=-1).fit(X_train, y_train)
#    m_gb = GradientBoostingRegressor().fit(X_train, y_train)
    m_mlp = MLPRegressor(hidden_layer_sizes=80, activation='relu', solver='adam', alpha=0.0001).fit(X_train, y_train)

    # Make predictions
    fday = datetime(2016, 5, 23) 
    max_lags = 15
    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        tst = this_test[(this_test.date >= day - timedelta(days=max_lags)) & (this_test.date <= day)].copy()
        tst = feat_eng(tst)
        tst = tst.fillna(0)
        
#        this_test = predict_sales(this_test, m_linreg, linreg_train_cols, 'preds_LinearReg', tst, day)
#        this_test = predict_sales(this_test, m_rf, train_cols, 'preds_RandomForest', tst, day)
#        this_test = predict_sales(this_test, m_gb, train_cols, 'preds_GradeintBoosting', tst, day)
        this_test = predict_sales(this_test, m_mlp, train_cols, 'preds_MultiLayerPerceptron', tst, day)
        

    test_final = this_test.loc[this_test.date >= fday]
    sub_final = pd.concat([sub_final, test_final])

In [75]:
from sklearn.neural_network import MLPRegressor

In [ ]:
sub_final.tail()

In [70]:
# Reframe for submission  

linreg_final = sub_final[['id','date', 'preds_LinearReg']]
linreg_final= linreg_final.pivot(index="id", columns="date", values="preds_LinearReg").reset_index()
linreg_final = linreg_final.rename_axis(None, axis=1)
linreg_final.head()

#linreg_final.to_csv('linreg_result.csv')

,id,2016-05-23 00:00:00,2016-05-24 00:00:00,2016-05-25 00:00:00,2016-05-26 00:00:00,2016-05-27 00:00:00,2016-05-28 00:00:00,2016-05-29 00:00:00,2016-05-30 00:00:00,2016-05-31 00:00:00,2016-06-01 00:00:00,2016-06-02 00:00:00,2016-06-03 00:00:00,2016-06-04 00:00:00,2016-06-05 00:00:00,2016-06-06 00:00:00,2016-06-07 00:00:00,2016-06-08 00:00:00,2016-06-09 00:00:00,2016-06-10 00:00:00,2016-06-11 00:00:00,2016-06-12 00:00:00,2016-06-13 00:00:00,2016-06-14 00:00:00,2016-06-15 00:00:00,2016-06-16 00:00:00,2016-06-17 00:00:00,2016-06-18 00:00:00,2016-06-19 00:00:00
0,HOBBIES_2_001_CA_3_validation,0.122208,0.123183,0.124158,0.125133,0.126108,0.120258,0.129616,0.133597,0.134572,0.140507,0.141482,0.142457,0.136608,0.126194,0.127169,0.128144,0.129119,0.130094,0.131069,0.125219,0.126194,0.127169,0.128144,0.129119,0.130094,0.131069,0.125219,0.126194
1,HOBBIES_2_002_CA_3_validation,0.168892,0.137143,0.139494,0.107745,0.075997,0.061889,0.064240,0.066591,0.068943,0.072063,0.074414,0.076765,0.062657,0.065009,0.067360,0.069711,0.072063,0.074414,0.076765,0.062657,0.065009,0.067360,0.069711,0.072063,0.074414,0.076765,0.062657,0.065009
2,HOBBIES_2_003_CA_3_validation,0.882057,0.805289,0.824977,0.763826,0.735287,0.995517,0.926902,0.857943,0.821250,0.756768,0.720075,0.667421,0.855654,0.787039,0.750346,0.713654,0.676961,0.640269,0.603576,0.823731,0.787039,0.750346,0.713654,0.676961,0.640269,0.603576,0.823731,0.787039
3,HOBBIES_2_004_CA_3_validation,0.363119,0.358456,0.353793,0.349130,0.344467,0.220905,0.216242,0.211579,0.206916,0.192070,0.187407,0.182744,0.210723,0.206060,0.201397,0.196733,0.192070,0.187407,0.182744,0.210723,0.206060,0.201397,0.196733,0.192070,0.187407,0.182744,0.210723,0.206060
4,HOBBIES_2_005_CA_3_validation,0.122093,0.123106,0.124118,0.125131,0.126143,0.120068,0.121081,0.122093,0.123106,0.127058,0.128070,0.129083,0.123008,0.124020,0.125033,0.126045,0.127058,0.128070,0.129083,0.123008,0.124020,0.125033,0.126045,0.127058,0.128070,0.129083,0.123008,0.124020
